In [ ]:
import ipywidgets.widgets as widgets
import sys
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.patches as patches
from matplotlib.transforms import Affine2D
from io import BytesIO

sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.utils import figsize_from_image_size, keep_rps

In [ ]:
image = widgets.Image(format='jpeg', width=600, height=600)
display(image)

In [ ]:
def show_detections(image, detections):
    fig = Figure(figsize=figsize_from_image_size(image))
    canvas = FigureCanvas(fig)
    ax = fig.gca()

    for det in detections:
        x0 = -max(det['top_left'][0], det['bottom_right'][0])
        y0 = -max(det['top_left'][1], det['bottom_right'][1])
        h = abs(det['bottom_right'][1] - det['top_left'][1])
        w = abs(det['bottom_right'][0] - det['top_left'][0])
        rect = patches.Rectangle(
            (x0, y0), w, h,
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)

        ax.text(
            x0, y0,
            '{} ({:0.0f}%)'.format(det['object_class'], det['confidence'] * 100.),
            fontsize=10, color='r'
        )

    ax.set_xlim(-1., 0.)
    ax.set_ylim(-1., 0.)
    ax.grid(which='both', linestyle='--', alpha=0.5)

    canvas.draw()
    jpeg = BytesIO()
    canvas.print_jpg(jpeg)
    image.value = jpeg.getvalue()

In [ ]:
from ballsbot_cpp import ballsbot_cpp

In [ ]:
%%time
ballsbot_cpp.startup_detection()

In [ ]:
ts = None
while True:
    ts = keep_rps(ts, fps=2)
    detections = ballsbot_cpp.detect()
    show_detections(image, detections)